In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer

class CustomDataset(Dataset):
    def __init__(self, texts, tags, tokenizer, tag2id, max_len=512):
        """
        Инициализация датасета для NER.
        
        :param texts: Список текстов (строк).
        :param tags: Список списков меток для каждого текста.
        :param tokenizer: Токенизатор BERT.
        :param tag2id: Словарь соответствия меток числовым идентификаторам.
        :param max_len: Максимальная длина последовательности.
        """
        self.texts = texts
        self.tags = tags
        self.tokenizer = tokenizer
        self.tag2id = tag2id
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tags = self.tags[idx]

        # Токенизация с возвратом отсутсвия (offset) для меток
        encoding = self.tokenizer(
            text,
            return_offsets_mapping=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len
        )

        # Инициализация меток для токенов
        labels = [self.tag2id.get(tag, self.tag2id['O']) for tag in tags]

        # Выравнивание меток по токенам
        offset_mappings = encoding.pop("offset_mapping")
        aligned_labels = []
        previous_word_idx = None

        for offset in offset_mappings:
            if offset[0] == 0 and offset[1] != 0:
                # Начало нового слова
                if previous_word_idx is None:
                    label = labels[0]
                else:
                    label = labels[previous_word_idx]
                previous_word_idx = previous_word_idx + 1 if previous_word_idx is not None else 0
            else:
                # Продолжение предыдущего слова или паддинг
                label = self.tag2id['O']
            aligned_labels.append(label)

        # Обработка случаев, когда меток меньше, чем токенов
        if len(aligned_labels) > self.max_len:
            aligned_labels = aligned_labels[:self.max_len]
        else:
            aligned_labels += [self.tag2id['O']] * (self.max_len - len(aligned_labels))

        return {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(aligned_labels, dtype=torch.long)
        }

...1. Параметры конструктора:
    - texts: Список строк, каждая из которых представляет собой текст для NER.
    - tags: Список списков меток, соответствующих каждому тексту. Метки могут быть в формате BIO/BIOES.
    - tokenizer: Токенизатор BERT для преобразования текста в токены.
    - tag2id: Словарь, сопоставляющий метки с числовыми идентификаторами (например, {'O': 0, 'B-PER': 1, 'I-PER': 2, ...}).
    - max_len: Максимальная длина последовательности (обычно 512 для BERT).

2. Метод __getitem__:
    - Токенизирует текст с использованием метода tokenizer, возвращая input_ids, attention_mask и offset_mapping для выравнивания меток по токенам.


- Инициализирует метки для токенов, сопоставляя исходные метки с токенами. Если один токен разбивается на несколько субтокенов, оставшиеся субтокены метятся как O.
    - Паддит или усекает последовательность меток до максимальной длины.
...

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import BertForTokenClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

class BertNER:
    def __init__(self, model_path, tokenizer_path, tag2id, id2tag, n_classes=None, epochs=3, model_save_path='bert_ner.pt'):
        """
        Инициализация модели NER на основе BERT.
        
        :param model_path: Путь или имя предобученной модели BERT.
        :param tokenizer_path: Путь или имя токенизатора BERT.
        :param tag2id: Словарь меток к идентификаторам.
        :param id2tag: Обратный словарь от идентификаторов к меткам.
        :param n_classes: Количество классов (если None, вывод берётся из tag2id).
        :param epochs: Количество эпох обучения.
        :param model_save_path: Путь для сохранения модели.
        """
        self.model = BertForTokenClassification.from_pretrained(
            model_path,
            num_labels=n_classes if n_classes is not None else len(tag2id),
            output_hidden_states=False
        )
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model_save_path = model_save_path
        self.max_len = 512
        self.epochs = epochs
        self.tag2id = tag2id
        self.id2tag = id2tag
        self.model.to(self.device)

    def prepare_data(self, X_train, y_train, X_valid, y_valid, batch_size=16):
        """
        Подготовка данных для обучения и валидации.
        
        :param X_train: Список тренировочных текстов.
        :param y_train: Список меток для тренировочных текстов.
        :param X_valid: Список валидационных текстов.
        :param y_valid: Список меток для валидационных текстов.
        :param batch_size: Размер батча.
        """
        self.train_set = CustomDataset(X_train, y_train, self.tokenizer, self.tag2id, max_len=self.max_len)
        self.valid_set = CustomDataset(X_valid, y_valid, self.tokenizer, self.tag2id, max_len=self.max_len)
        
        self.train_loader = DataLoader(self.train_set, batch_size=batch_size, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=batch_size, shuffle=False)
        
        self.optimizer = AdamW(self.model.parameters(), lr=3e-5, correct_bias=False)
        total_steps = len(self.train_loader) * self.epochs
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=int(0.1 * total_steps),
            num_training_steps=total_steps
        )
        self.loss_fn = torch.nn.CrossEntropyLoss(ignore_index=self.tag2id['O']).to(self.device)

    def train_epoch(self):
        """
        Обучение модели на одной эпохе.
        
        :return: Среднее значение потерь и точность на тренировочном наборе.
        """
        self.model.train()
        losses = []
        correct_predictions = 0
        total_predictions = 0

        for batch in tqdm(self.train_loader, desc="Training"):
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits
                        losses.append(loss.item())

            # Предсказания
            preds = torch.argmax(logits, dim=2)

            # Игнорирование специальных токенов (например, [PAD], [CLS], [SEP])
            active_loss = labels.view(-1) != self.tag2id['O']
            active_preds = preds.view(-1)[active_loss]
            active_labels = labels.view(-1)[active_loss]

            correct_predictions += torch.sum(active_preds == active_labels)
            total_predictions += torch.sum(active_loss).item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()

        avg_loss = np.mean(losses)
        accuracy = correct_predictions.double() / total_predictions
        return avg_loss, accuracy.item()

    def eval_epoch(self):
        """
        Оценка модели на валидационном наборе.
        
        :return: Среднее значение потерь и точность на валидационном наборе.
        """
        self.model.eval()
        losses = []
        correct_predictions = 0
        total_predictions = 0

        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in tqdm(self.valid_loader, desc="Validation"):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                logits = outputs.logits

                losses.append(loss.item())

                preds = torch.argmax(logits, dim=2)

                active_loss = labels.view(-1) != self.tag2id['O']
                active_preds = preds.view(-1)[active_loss]
                active_labels = labels.view(-1)[active_loss]

                correct_predictions += torch.sum(active_preds == active_labels)
                total_predictions += torch.sum(active_loss).item()

                # Собираем все предсказания и метки для отчёта
                all_preds.extend(active_preds.cpu().numpy())
                all_labels.extend(active_labels.cpu().numpy())

        avg_loss = np.mean(losses)
        accuracy = correct_predictions.double() / total_predictions
        report = classification_report(all_labels, all_preds, target_names=[self.id2tag[id] for id in sorted(self.id2tag)])
        return avg_loss, accuracy.item(), report

    def fit(self):
        """
        Обучение модели на всем наборе данных.
        """
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_loss, train_acc = self.train_epoch()
            print(f'Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}')
            
            val_loss, val_acc, report = self.eval_epoch()
            print(f'Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}')
            print("Classification Report:")
            print(report)
            
            # Сохранение модели после каждой эпохи (по желанию)
            self.save_model(f'{self.model_save_path}_epoch_{epoch + 1}.pt')

    def save_model(self, path=None):
        """
        Сохранение модели.
        
        :param path: Путь для сохранения модели. Если None, используется self.model_save_path.
        """
        if path is None:
            path = self.model_save_path
        self.model.save_pretrained(path)
        self.tokenizer.save_pretrained(path)

    def load_model(self, path):
        """
        Загрузка модели из сохраненного состояния.
        
        :param path: Путь к сохраненной модели.
        """
        self.model = BertForTokenClassification.from_pretrained(path)
        self.tokenizer = BertTokenizer.from_pretrained(path)
        self.model.to(self.device)

1. Использование BertForTokenClassification:
    - Модель BertForTokenClassification специально предназначена для задач токен-классификации, таких как NER. Она автоматически добавляет классификационный слой поверх BERT.

2. Параметры Конструктора:
    - tag2id и id2tag используются для преобразования меток в числовые идентификаторы и обратно.
    - n_classes устанавливает количество классов, основываясь на tag2id, если не задано явно.

3. Метод prepare_data:
    - Создает экземпляры CustomDataset для тренировочных и валидационных данных.
    - Инициализирует DataLoader с указанным размером батча.
    - Настраивает оптимизатор AdamW и планировщик (scheduler) для изменения скорости обучения.
    - Использует CrossEntropyLoss с игнорированием метки O (можно настроить по необходимости).

4. Методы train_epoch и eval_epoch:
    - trainepoch**:
        - Переводит модель в режим обучения.
        - Проходит по всем батчам тренировочного набора.
        - Выполняет прямой и обратный проход, обновляя веса модели.
        - Считает средние потери и точность на тренировочном наборе.
    - **evalepoch:
        - Переводит модель в режим оценки.
        - Проходит по всем батчам валидационного набора без обновления весов.
        - Считает средние потери и точность на валидационном наборе.
        - Генерирует отчёт о классификации (classification_report) с использованием метрик из sklearn.

5. Метод fit:
    - Обучает модель на заданное количество эпох.
    - После каждой эпохи выводит информацию об обучении и валидации.
    - Опционально сохраняет модель после каждой эпохи.

6. Методы save_model и load_model:
    - Позволяют сохранять и загружать состояние модели для дальнейшего использования.

## 3. Пример использования классов для NER

Ниже приведён пример того, как использовать обновлённые классы CustomDataset и BertNER для обучения модели NER.


Шаг 1: Подготовка данных

Предположим, у вас есть данные в формате:


In [ ]:
X_train = ["John lives in New York.", "Mary works at Google."]
y_train = [["B-PER", "O", "O", "B-LOC", "I-LOC", "O"],
           ["B-PER", "O", "O", "B-ORG", "O"]]

X_valid = ["Alice moved to Los Angeles."]
y_valid = [["B-PER", "O", "O", "B-LOC", "I-LOC", "O"]]

Определение словарей меток



In [2]:
tags = list(set(tag for doc in y_train + y_valid for tag in doc))
tag2id = {tag: idx for idx, tag in enumerate(tags)}
id2tag = {idx: tag for tag, idx in tag2id.items()}

NameError: name 'y_train' is not defined

 Инициализация и подготовка модели


In [ ]:
# Инициализация модели
bert_ner = BertNER(
    model_path='bert-base-uncased',
    tokenizer_path='bert-base-uncased',
    tag2id=tag2id,
    id2tag=id2tag,
    n_classes=len(tag2id),
    epochs=3,
    model_save_path='bert_ner_model'
)

# Подготовка данных
bert_ner.prepare_data(X_train, y_train, X_valid, y_valid, batch_size=16)

...## 4. Дополнительные замечания

1. Обработка субтокенов:
    - В реализации CustomDataset субтокены помечаются как O. Это простой подход, но можно улучшить его, например, теряя метки для субтокенов или используя специальные метки для продолжения сущностей.

2. Выбор метрики:
    - Для NER часто используются метрики Precision, Recall и F1-score для каждой сущности. classification_report из sklearn помогает в этом.

3. Использование более продвинутых токенизаторов:
    - Если вы используете токенизаторы, которые не возвращают offset_mapping, вам потребуется другой способ выравнивания меток.

4. Инициализация меток:
    - Убедитесь, что ваши метки правильно соответствуют токенам. Ошибки в выравнивании могут существенно снизить качество модели.

5. Расширенные стратегии обучение:


- Можно внедрить дополнительные техники, такие как ранняя остановка (early stopping), увеличение данных (data augmentation) и т.д., для повышения качества модели.

## Заключение

Переписав классы CustomDataset и BertClassifier1 для задачи NER, мы адаптировали их под токен-классификацию, корректно выравнивая метки по токенам и используя специализированную модель BertForTokenClassification. Этот подход позволит эффективно обучать модель на задачах распознавания именованных сущностей, обеспечивая точные предсказания на уровне токенов.

In [ ]:
# Сохранение модели
bert_ner.save_model('bert_ner_final.pt')

# Загрузка модели
bert_ner.load_model('bert_ner_final.pt')